**Finetuning of Model**

In [2]:
!pip install transformers datasets
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import Dataset
import numpy as np
from sklearn.metrics import accuracy_score

df = pd.read_csv('/content/Untitled spreadsheet - Sheet4.csv')

df['sentence'] = df['sentence'].str.lower()

unique_labels = df['label'].unique()
label_to_id = {label: idx for idx, label in enumerate(unique_labels)}
df['label_id'] = df['label'].map(label_to_id)
num_labels = len(unique_labels)

train_df, test_df = train_test_split(df, test_size=0.2, stratify=df['label'], random_state=42)

train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

tokenizer = AutoTokenizer.from_pretrained("EleutherAI/pythia-70m")
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

def tokenize_function(examples):
    return tokenizer(examples['sentence'], padding="max_length", truncation=True, max_length=128)

train_tokenized = train_dataset.map(tokenize_function, batched=True)
test_tokenized = test_dataset.map(tokenize_function, batched=True)

train_tokenized = train_tokenized.remove_columns(['sentence', 'label', '__index_level_0__'])
train_tokenized = train_tokenized.rename_column("label_id", "labels")
train_tokenized.set_format("torch")

test_tokenized = test_tokenized.remove_columns(['sentence', 'label', '__index_level_0__'])
test_tokenized = test_tokenized.rename_column("label_id", "labels")
test_tokenized.set_format("torch")

model = AutoModelForSequenceClassification.from_pretrained(
    "EleutherAI/pythia-70m",
    num_labels=num_labels
)
model.config.pad_token_id = tokenizer.pad_token_id

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=1.5e-5,
    per_device_train_batch_size=5,
    per_device_eval_batch_size=3,
    num_train_epochs=5,
    weight_decay=0.02,
    logging_steps=10,
    logging_dir='./logs',
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized,
    eval_dataset=test_tokenized,
)

trainer.train()

eval_results = trainer.evaluate()
print("Evaluation results:", eval_results)

predictions = trainer.predict(test_tokenized)
preds = np.argmax(predictions.predictions, axis=1)
true_labels = predictions.label_ids
accuracy = accuracy_score(true_labels, preds)
print(f"Accuracy: {accuracy:.4f}")

model.save_pretrained("./Tifin/trained_model")
tokenizer.save_pretrained("./Tifin/trained_model")
print("Model saved to ./Tifin/trained_model")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 17.5 MB/s eta 0:00:00


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/396 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

...

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: abhaysasthas (abhaysasthas-vellore-institute-of-technology) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss
1,0.896700,0.807672
2,0.429400,0.523169
3,0.219000,0.573772
4,0.000100,0.450670
5,0.000000,0.452976


Evaluation results: {'eval_loss': 0.4506700932979584, 'eval_runtime': 0.6252, 'eval_samples_per_second': 348.663, 'eval_steps_per_second': 116.754, 'epoch': 5.0}
Accuracy: 0.9266
Model saved to ./Tifin/trained_model


Epoch	Training Loss	Validation Loss

1.308300	1.527181

0.547400	0.622491

0.106500	0.469147

0.023500	0.407750

0.001000	0.428317


Evaluation results: {'eval_loss': 0.4077504575252533, 'eval_runtime': 0.4685, 'eval_samples_per_second': 465.282, 'eval_steps_per_second': 93.91, 'epoch': 5.0}

Accuracy: 0.9128


learning_rate=1e-5,
per_device_train_batch_size=5,    
per_device_eval_batch_size=5,     
num_train_epochs=5,               
weight_decay=0.01,
logging_steps=10,  



---


learning_rate=1.5e-5,                
    per_device_train_batch_size=5,    
    per_device_eval_batch_size=3,     
    num_train_epochs=5,                
    weight_decay=0.02,                 
    logging_steps=10,  

Epoch	Training Loss	Validation Loss

1	0.883400	0.816362

2	0.509100	0.401338

3	0.090700	0.315920

4	0.003300	0.312922

5	0.000000	0.307967

Evaluation results: {'eval_loss': 0.30796730518341064, 'eval_runtime': 0.7502, 'eval_samples_per_second': 290.59, 'eval_steps_per_second': 97.308, 'epoch': 5.0}

Accuracy: 0.9358

In [ ]:
#from huggingface_hub import notebook_login
#notebook_login()

In [ ]:
#model.push_to_hub("abhaysastha/intent-model-70m-ft")
#tokenizer.push_to_hub("abhaysastha/intent-model-70m-ft")

adapter_model.safetensors:   0%|          | 0.00/438k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/abhaysastha/intent-model-70m-ft/commit/2f8cd947cf5f6b039662b0672e48aefa8f989b00', commit_message='Upload tokenizer', commit_description='', oid='2f8cd947cf5f6b039662b0672e48aefa8f989b00', pr_url=None, repo_url=RepoUrl('https://huggingface.co/abhaysastha/intent-model-70m-ft', endpoint='https://huggingface.co', repo_type='model', repo_id='abhaysastha/intent-model-70m-ft'), pr_revision=None, pr_num=None)

**Local inferencing**

In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("./Tifin/trained_model")
tokenizer = AutoTokenizer.from_pretrained("./Tifin/trained_model")

id_to_label = {0:"100_NIGHT_TRIAL_OFFER",1:"ABOUT_SOF_MATTRESS",2:"CANCEL_ORDER",3:"CHECK_PINCODE",4:"COD",5:"COMPARISON",6:"DELAY_IN_DELIVERY",
          7:"DISTRIBUTORS",8:"EMI",9:"ERGO_FEATURES",10:"LEAD_GEN",11:"MATTRESS_COST",12:"OFFERS",13:"ORDER_STATUS",14:"ORTHO_FEATURES",15:"PILLOWS",
          16:"PRODUCT_VARIANTS",17:"RETURN_EXCHANGE",18:"SIZE_CUSTOMIZATION",19:"WARRANTY",20:"WHAT_SIZE_TO_ORDER"}

def infer_intent(text):

    model.eval()

    inputs = tokenizer(
        text,
        return_tensors="pt",
        padding="max_length",
        truncation=True,
        max_length=128
    )

    with torch.no_grad():
        outputs = model(**inputs)

    predicted_id = torch.argmax(outputs.logits, dim=-1).item()

    return id_to_label.get(predicted_id, "Unknown")

if __name__ == "__main__":
    print("Enter sentence ('exit'for quit):")
    while True:
        user_input = input(">> ")
        if user_input.lower() == "exit":
            break

        predicted_intent = infer_intent(user_input.lower())
        print("Predicted Intent Label:", predicted_intent)


Enter a sentence to predict its intent label (type 'exit' to quit):
>> exit


# **DEPLOY**

In [4]:
!curl ipv4.icanhazip.com
!npm install localtunnel
!pip install streamlit transformers peft bitsandbytes torch numpy


34.16.134.4
⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇
added 22 packages in 4s
⠇
⠇3 packages are looking for funding
⠇  run `npm fund` for details
⠇Collecting streamlit
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 65.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 49.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 38.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [5]:
%%writefile app.py

import streamlit as st
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("./Tifin/trained_model")
tokenizer = AutoTokenizer.from_pretrained("./Tifin/trained_model")

id_to_label = {
    0: "100_NIGHT_TRIAL_OFFER",
    1: "ABOUT_SOF_MATTRESS",
    2: "CANCEL_ORDER",
    3: "CHECK_PINCODE",
    4: "COD",
    5: "COMPARISON",
    6: "DELAY_IN_DELIVERY",
    7: "DISTRIBUTORS",
    8: "EMI",
    9: "ERGO_FEATURES",
    10: "LEAD_GEN",
    11: "MATTRESS_COST",
    12: "OFFERS",
    13: "ORDER_STATUS",
    14: "ORTHO_FEATURES",
    15: "PILLOWS",
    16: "PRODUCT_VARIANTS",
    17: "RETURN_EXCHANGE",
    18: "SIZE_CUSTOMIZATION",
    19: "WARRANTY",
    20: "WHAT_SIZE_TO_ORDER"
}

def infer_intent(text):
    model.eval()

    inputs = tokenizer(
        text,
        return_tensors="pt",
        padding="max_length",
        truncation=True,
        max_length=128
    )

    with torch.no_grad():
        outputs = model(**inputs)

    predicted_id = torch.argmax(outputs.logits, dim=-1).item()
    return id_to_label.get(predicted_id, "Unknown")

def main():
    st.title("Intent Classification App")
    st.write("Enter a sentence to predict its intent label:")

    user_input = st.text_input("Your sentence:")

    if st.button("Predict"):
        if user_input.strip() != "":
            prediction = infer_intent(user_input.lower())
            st.success(f"Predicted Intent Label: **{prediction}**")
        else:
            st.error("Please enter a sentence to predict the intent.")

if __name__ == "__main__":
    main()


Writing app.py


In [2]:
!curl ipv4.icanhazip.com
print("^password")

34.127.89.21
^password


In [10]:
!streamlit run app.py &>./logs.txt & npx localtunnel --port 8501
print("^click the above link to run streamlit app")

⠙your url is: https://olive-rockets-argue.loca.lt
exit
^C
